In [9]:
import csv
from datetime import datetime
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Define the stations with their labels and coordinates
stations = [
    {"label": "VSKP", "latitude": 23.125, "longitude": 80},
    {"label": "DVD", "latitude": 17.0150, "longitude": 82.2321},
    {"label": "AKP", "latitude": 16.9849, "longitude": 81.7866},
    # Add all other stations with their latitudes and longitudes...
]

# Example raw data with train delays for each date
data = [
    [datetime(2023, 12, 18), 1, 0, 2],  # Example delays for VSKP, DVD, AKP
    [datetime(2023, 12, 19), 0, 1, 3],
    # Add more rows as needed
]

# Prepare the initial table with just dates and delays
dates = [row[0].strftime('%d/%m/%y') for row in data]
df = pd.DataFrame({"Date": dates})

# Add delay data for each station in the format specified
for i, station in enumerate(stations):
    df[station["label"]] = [row[i + 1] for row in data]

# Define columns in the required order and retrieve weather data
for station in stations:
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": station["latitude"],
        "longitude": station["longitude"],
        "start_date": "2023-12-18",
        "end_date": "2023-12-19",  # Adjusted end date based on data range
        "daily": ["weather_code", "temperature_2m_mean", "wind_speed_10m_max"]
    }

    # Fetch data from API
    response = openmeteo.weather_api(url, params=params)[0]
    daily = response.Daily()
    
    dates = pd.date_range(
        start=pd.to_datetime(daily.Time(), unit="s", utc=True),
        end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=daily.Interval()),
        inclusive="left"
    )

    # Ensure dates is a list, matching the length of weather data variables
    dates = list(dates)
    weather_code = daily.Variables(0).ValuesAsNumpy()
    temperature = daily.Variables(1).ValuesAsNumpy()
    wind_speed = daily.Variables(2).ValuesAsNumpy()

    # Create a temporary DataFrame to store the data for this station
    temp_df = pd.DataFrame({
        "Date": [date.strftime('%d/%m/%y') for date in dates],
        f"{station['label']}_Delay": df[station["label"]].values,
        f"weather_code_{station['label']}": weather_code,
        f"Temperature_{station['label']}": temperature,
        f"Wind_Speed_{station['label']}": wind_speed,
        f"Distance_travelled_{station['label']}": [None] * len(weather_code)  # Placeholder for Distance
    })

    # Merge this station's data back into the main DataFrame
    df = df.merge(temp_df, on="Date", how="left")

# Drop the original delay columns used for initialization
df.drop(columns=[station["label"] for station in stations], inplace=True)

# Save final table with delays and weather details
df.to_csv("test.csv", index=False)
print("Data has been formatted and saved to 'merged.csv'")


Data has been formatted and saved to 'merged.csv'


In [10]:
import csv
from datetime import datetime
import openmeteo_requests
import requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup API key and base URL for Indian Railway API
API_KEY = '4ca75cf9c1cb385e61c3a856ae817766'
BASE_URL = "http://indianrailapi.com/api/v2/StationCodeToName/apikey/{}/StationCode/".format(API_KEY)

def fetch_station_details(station_code):
    url = BASE_URL + station_code
    response = requests.get(url)
    data = response.json()
    # print(data)
    if data['ResponseCode'] == "200" and data['Status'] == "SUCCESS" and station_code not in ["NDPM", "RKMP"]:
        latitude = data['Station']['Latitude']
        longitude = data['Station']['Longitude']
        return {
            "label": station_code,
            "latitude": float(latitude) if latitude else None,
            "longitude": float(longitude) if longitude else None
        }
    elif station_code == "NDPM":
        return {
            "label": station_code,
            "latitude": float("22.75461"),
            "longitude":  float("77.7169022")
        } 
    elif station_code == "RKMP":
        return {
            "label": station_code,
            "latitude": float("23.3"),
            "longitude": float("79.7")
        }

    else:
        return None


# List of station codes for which you want to fetch details
station_codes = ['JBP', 'MML', 'SRID','NU', 'KY', 'GAR','PPI', 'ET', 'NDPM', 'RKMP']  # Add more station codes as needed

# Create list of station details
stations = []
for code in station_codes:
    station_info = fetch_station_details(code)
    if station_info:
        stations.append(station_info)
    else:
        print(f"Failed to fetch details for station code: {code}")

# Set up the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

data = [
    [ datetime(2023,12,18),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2023,12,19),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2023,12,20),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2023,12,21),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2023,12,22),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2023,12,23),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2023,12,24),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2023,12,25),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2023,12,26),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2023,12,27),25,30,41,47,54,52,49,51,50,49,49,50,50,10]
,[ datetime(2023,12,28),0,4,29,45,50,52,54,56,56,57,58,55,55,10]
,[ datetime(2023,12,29),23,26,28,32,35,35,38,38,38,38,36,30,30,2]
,[ datetime(2023,12,30),0,4,24,31,36,37,38,40,40,40,40,50,50,5]
,[ datetime(2023,12,31),0,4,17,21,24,27,27,27,25,28,29,40,40,-2]
,[ datetime(2024,1,1),0,2,0,5,7,9,11,12,11,10,9,24,24,-25]
,[ datetime(2024,1,2),4,4,20,24,31,33,34,34,35,36,34,50,50,0]
,[ datetime(2024,1,3),1,6,2,9,20,21,22,23,23,25,24,40,40,-5]
,[ datetime(2024,1,4),0,9,16,27,31,33,35,34,34,34,33,45,45,0]
,[ datetime(2024,1,5),22,32,27,29,34,36,35,40,40,47,42,45,45,5]
,[ datetime(2024,1,6),1,3,0,4,4,5,8,11,11,10,11,29,29,-19]
,[ datetime(2024,1,7),1,21,17,24,28,31,32,35,35,35,37,50,50,0]
,[ datetime(2024,1,8),1,0,2,27,34,35,36,38,37,39,40,50,50,0]
,[ datetime(2024,1,9),0,4,0,2,4,4,5,6,9,8,9,20,20,-27]
,[ datetime(2024,1,10),2,7,2,37,52,54,54,58,59,59,60,65,65,14]
,[ datetime(2024,1,11),1,3,4,8,11,13,15,17,21,21,23,35,35,-16]
,[ datetime(2024,1,12),0,2,0,0,1,1,0,0,0,0,1,10,10,-38]
,[ datetime(2024,1,13),0,4,0,7,37,39,39,40,40,39,39,45,45,0]
,[ datetime(2024,1,14),0,4,0,3,17,18,20,19,20,19,29,37,37,-10]
,[ datetime(2024,1,15),0,22,26,36,38,37,38,40,36,37,38,47,47,0]
,[ datetime(2024,1,16),0,5,2,24,29,33,35,38,37,38,38,50,50,5]
,[ datetime(2024,1,17),0,10,4,21,23,24,26,25,28,27,28,40,40,-5]
,[ datetime(2024,1,18),0,19,16,23,30,32,31,45,45,46,48,41,41,-6]
,[ datetime(2024,1,19),0,2,5,14,16,18,17,16,16,14,16,27,27,-20]
,[ datetime(2024,1,20),3,4,22,22,25,27,27,25,24,24,23,40,40,-15]
,[ datetime(2024,1,21),0,8,4,10,13,13,14,16,15,15,16,35,35,-10]
,[ datetime(2024,1,22),0,12,8,41,52,53,52,51,50,46,48,48,48,0]
,[ datetime(2024,1,23),0,19,17,22,27,26,29,29,29,29,30,40,40,-5]
,[ datetime(2024,1,24),0,4,10,24,35,37,37,37,38,38,43,50,50,0]
,[ datetime(2024,1,25),0,3,0,5,9,10,10,24,23,24,24,30,30,-15]
,[ datetime(2024,1,26),0,3,0,1,5,5,4,4,4,5,6,25,25,-21]
,[ datetime(2024,1,27),1,4,13,14,15,15,14,13,13,11,11,22,22,-27]
,[ datetime(2024,1,28),0,5,0,1,3,3,4,5,5,5,5,25,25,-26]
,[ datetime(2024,1,29),0,3,0,3,5,4,6,5,4,14,13,40,40,-5]
,[ datetime(2024,1,30),0,5,3,6,13,17,22,24,25,27,27,45,45,0]
,[ datetime(2024,1,31),0,5,0,3,6,7,7,6,6,6,6,18,18,-27]
,[ datetime(2024,2,1),0,4,7,32,36,37,38,37,37,38,38,50,50,0]
,[ datetime(2024,2,2),0,4,0,3,15,17,17,15,16,18,19,25,25,-23]
,[ datetime(2024,2,3),0,5,11,12,17,18,22,22,24,25,25,40,40,-5]
,[ datetime(2024,2,4),0,5,1,4,6,7,8,8,8,7,11,30,30,-25]
,[ datetime(2024,2,5),0,4,1,1,4,4,5,4,5,7,7,30,30,0]
,[ datetime(2024,2,6),0,4,4,7,37,39,37,37,45,49,49,45,45,-2]
,[ datetime(2024,2,7),0,3,0,3,7,7,8,9,17,19,18,30,30,-17]
,[ datetime(2024,2,8),0,3,10,14,16,17,17,17,18,18,18,30,30,-15]
,[ datetime(2024,2,9),0,4,18,32,34,35,34,33,34,36,35,44,44,-5]
,[ datetime(2024,2,10),1,4,26,29,33,34,33,34,32,29,31,50,50,5]
,[ datetime(2024,2,11),0,5,0,11,15,15,17,18,19,20,21,35,35,-14]
,[ datetime(2024,2,12),0,4,9,11,15,18,19,19,19,20,21,33,33,-20]
,[ datetime(2024,2,13),0,40,36,59,67,71,71,72,74,78,79,70,70,15]
,[ datetime(2024,2,14),0,4,28,29,33,35,33,32,31,30,32,45,45,0]
,[ datetime(2024,2,15),0,4,14,14,14,16,18,19,20,19,20,35,35,-12]
,[ datetime(2024,2,16),0,4,6,10,38,42,42,44,43,42,42,45,45,0]
,[ datetime(2024,2,17),0,4,28,51,53,57,58,57,56,55,55,64,64,14]
,[ datetime(2024,2,18),0,3,4,6,8,14,17,16,13,12,13,25,25,-20]
,[ datetime(2024,2,19),0,4,22,25,28,32,33,32,32,31,31,40,40,-8]
,[ datetime(2024,2,20),0,2,7,16,19,23,22,22,21,21,21,30,30,-7]
,[ datetime(2024,2,21),0,4,47,45,50,49,48,47,45,44,52,55,55,15]
,[ datetime(2024,2,22),0,4,17,18,28,29,30,30,27,28,27,40,40,-10]
,[ datetime(2024,2,23),0,4,0,7,9,13,15,24,24,24,24,40,40,-2]
,[ datetime(2024,2,24),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,2,25),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,2,26),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,2,27),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,2,28),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,2,29),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,3,1),0,3,0,5,11,12,11,13,13,21,20,38,38,-8]
,[ datetime(2024,3,2),0,3,0,5,20,21,22,20,21,21,22,45,45,0]
,[ datetime(2024,3,3),0,4,22,53,59,68,71,70,82,81,83,80,79,33]
,[ datetime(2024,3,4),0,4,12,29,29,29,35,34,38,38,36,45,45,0]
,[ datetime(2024,3,5),0,4,12,34,39,39,39,38,38,38,38,45,45,0]
,[ datetime(2024,3,6),0,4,16,52,58,58,58,57,57,56,55,51,51,10]
,[ datetime(2024,3,7),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,3,8),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,3,9),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,3,10),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,3,11),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,3,12),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,3,13),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,3,14),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,3,15),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,3,16),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,3,17),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,3,18),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,3,19),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,3,20),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,3,21),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,3,22),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,3,23),0,4,37,49,54,54,55,54,54,53,52,55,55,10]
,[ datetime(2024,3,24),0,3,9,54,55,56,56,55,54,53,53,55,55,5]
,[ datetime(2024,3,25),0,0,0,12,15,28,31,32,33,33,41,47,47,15]
,[ datetime(2024,3,26),0,4,38,59,59,62,62,60,60,60,57,45,45,0]
,[ datetime(2024,3,27),2,0,11,25,25,28,28,29,30,28,29,45,45,-5]
,[ datetime(2024,3,28),3,3,1,6,7,8,9,8,22,8,8,25,25,-24]
,[ datetime(2024,3,29),0,3,0,1,4,5,7,5,6,6,8,30,30,-15]
,[ datetime(2024,3,30),0,4,8,22,26,27,31,28,30,31,31,40,40,0]
,[ datetime(2024,3,31),0,4,17,24,29,35,72,75,71,71,70,65,65,9]
,[ datetime(2024,3,1),0,47,51,53,55,56,56,56,66,67,97,100,100,44]
,[ datetime(2024,4,2),3,5,18,23,27,27,26,26,26,27,28,55,55,0]
,[ datetime(2024,4,3),0,4,26,43,46,48,48,46,46,44,42,50,50,0]
,[ datetime(2024,4,4),1,6,3,21,24,24,24,24,24,24,24,45,45,-1]
,[ datetime(2024,4,5),0,4,19,30,35,35,34,36,39,35,35,50,50,5]
,[ datetime(2024,4,6),0,2,0,5,7,8,9,8,6,18,16,26,26,-23]
,[ datetime(2024,4,7),0,5,2,7,10,12,12,12,12,13,18,12,14,19]
,[ datetime(2024,4,8),0,2,0,3,6,6,6,5,4,4,5,35,35,-10]
,[ datetime(2024,4,9),0,2,0,44,46,46,47,46,44,44,43,50,50,5]
,[ datetime(2024,4,10),0,3,0,0,1,2,1,2,4,8,13,40,40,-4]
,[ datetime(2024,4,11),0,4,5,9,13,15,15,16,17,16,53,60,60,10]
,[ datetime(2024,4,12),0,6,8,13,17,18,19,19,19,20,30,50,50,5]
,[ datetime(2024,4,13),2,3,13,18,18,17,17,16,15,15,16,34,34,-15]
,[ datetime(2024,4,14),0,4,0,15,14,16,16,17,16,16,17,40,40,-5]
,[ datetime(2024,4,15),0,4,7,15,17,20,20,21,20,19,19,41,41,-5]
,[ datetime(2024,4,16),0,4,1,2,4,5,5,4,4,4,22,40,40,0]
,[ datetime(2024,4,17),0,4,0,34,38,39,38,35,34,32,33,45,45,5]
,[ datetime(2024,4,18),37,50,42,50,53,60,63,67,76,79,95,90,90,40]
,[ datetime(2024,4,19),5,4,17,22,26,27,27,28,28,27,29,45,45,-4]
,[ datetime(2024,4,20),0,4,1,4,4,5,5,6,6,4,4,20,20,-21]
,[ datetime(2024,4,21),0,4,1,4,7,8,8,8,7,19,25,45,45,0]
,[ datetime(2024,4,22),0,5,11,13,20,20,21,24,32,34,48,55,55,5]
,[ datetime(2024,4,23),1,6,3,22,27,31,31,31,31,33,33,50,50,0]
,[ datetime(2024,4,24),0,4,0,3,9,8,8,8,10,9,13,30,30,-20]
,[ datetime(2024,4,25),0,3,1,17,23,25,39,47,47,49,52,59,59,5]
,[ datetime(2024,4,26),0,4,0,9,12,13,15,14,13,13,13,30,30,-15]
,[ datetime(2024,4,27),0,7,5,8,14,16,18,18,21,24,25,40,40,-5]
,[ datetime(2024,4,28),6,4,13,15,19,19,20,18,20,22,21,45,45,0]
,[ datetime(2024,4,29),1,4,0,0,4,6,6,6,5,3,3,30,30,-15]
,[ datetime(2024,4,30),0,2,10,23,28,36,45,47,49,49,120,163,163,108]
,[ datetime(2024,4,1),0,4,6,8,20,16,12,10,10,9,9,40,40,0]
,[ datetime(2024,5,2),0,4,22,41,46,49,51,52,53,53,55,65,65,15]
,[ datetime(2024,5,3),0,4,0,4,7,7,7,7,7,6,7,30,30,0]
,[ datetime(2024,5,4),0,4,9,15,19,20,20,20,19,19,34,64,64,15]
,[ datetime(2024,5,5),0,3,0,2,5,5,5,5,4,6,3,30,30,-15]
,[ datetime(2024,5,6),0,4,1,25,28,30,30,32,31,32,33,55,55,10]
,[ datetime(2024,5,7),0,4,13,19,22,25,26,26,29,30,30,55,55,0]
,[ datetime(2024,5,8),25,27,21,24,26,27,28,26,26,26,25,45,45,-4]
,[ datetime(2024,5,9),0,4,30,32,36,37,37,34,34,33,32,50,50,0]
,[ datetime(2024,5,10),31,33,31,41,46,47,50,51,52,52,53,65,65,13]
,[ datetime(2024,5,11),0,2,26,29,33,33,33,32,32,31,32,50,50,0]
,[ datetime(2024,5,12),5,10,14,17,21,26,23,23,23,24,23,48,48,0]
,[ datetime(2024,5,13),0,4,0,27,32,33,32,32,34,35,36,54,54,5]
,[ datetime(2024,5,14),1,4,41,61,67,77,74,71,72,72,73,65,65,15]
,[ datetime(2024,5,15),0,4,2,7,12,13,12,13,13,16,17,39,39,5]
,[ datetime(2024,5,16),0,4,12,32,41,42,42,44,45,47,47,40,40,14]
,[ datetime(2024,5,17),0,3,0,64,67,67,67,67,69,67,64,65,65,13]
,[ datetime(2024,5,18),0,3,0,17,19,21,20,21,21,26,27,55,55,12]
,[ datetime(2024,5,19),0,3,0,16,20,20,19,18,30,31,31,45,45,0]
,[ datetime(2024,5,20),0,5,23,47,54,55,57,56,56,55,56,77,77,15]
,[ datetime(2024,5,21),0,2,0,7,12,12,15,12,14,14,71,61,61,14]
,[ datetime(2024,5,22),0,3,0,29,34,32,32,31,29,31,28,40,40,-10]
,[ datetime(2024,5,23),0,4,20,47,55,57,74,75,75,75,75,56,56,14]
,[ datetime(2024,5,24),1,4,3,7,7,8,7,7,6,4,6,30,30,-20]
,[ datetime(2024,5,25),1,4,30,32,41,37,37,39,52,52,55,60,60,10]
,[ datetime(2024,5,26),0,4,49,53,57,59,61,61,60,56,51,55,55,12]
,[ datetime(2024,5,27),0,4,21,38,45,45,46,44,44,45,47,54,54,0]
,[ datetime(2024,5,28),6,4,18,26,28,29,29,27,26,26,27,45,45,0]
,[ datetime(2024,5,29),1,4,17,28,32,32,33,33,35,34,35,50,50,5]
,[ datetime(2024,5,30),1,4,0,19,21,21,22,22,22,29,31,45,45,0]
,[ datetime(2024,5,31),0,4,18,21,26,28,27,27,26,24,25,55,55,5]
,[ datetime(2024,5,1),0,4,1,9,13,13,15,13,11,8,7,25,25,-25]
,[ datetime(2024,6,2),0,7,3,41,46,46,46,47,47,46,44,55,55,10]
,[ datetime(2024,6,3),41,125,125,132,137,138,138,139,138,136,137,150,150,105]
,[ datetime(2024,6,4),2,20,20,36,44,47,48,47,47,47,48,45,45,0]
,[ datetime(2024,6,5),0,41,36,43,48,48,48,50,50,54,55,58,58,10]
,[ datetime(2024,6,6),2,4,8,27,34,35,35,36,37,37,37,50,50,11]
,[ datetime(2024,6,7),0,5,1,37,46,49,49,47,42,40,40,51,51,14]
,[ datetime(2024,6,8),64,67,66,72,79,88,83,79,98,99,99,99,99,43]
,[ datetime(2024,6,9),0,4,27,22,21,23,26,27,27,26,27,40,40,-10]
,[ datetime(2024,6,10),22,28,24,24,30,32,31,31,31,31,31,45,45,0]
,[ datetime(2024,6,11),0,11,6,10,21,22,21,21,23,25,26,45,45,0]
,[ datetime(2024,6,12),0,4,6,13,19,19,20,20,22,23,23,27,27,-3]
,[ datetime(2024,6,13),0,4,13,21,29,30,30,30,30,30,30,50,50,0]
,[ datetime(2024,6,14),0,21,17,19,25,27,27,27,27,27,26,16,16,-5]
,[ datetime(2024,6,15),4,3,4,4,10,10,12,9,9,8,8,20,20,-30]
,[ datetime(2024,6,16),0,4,5,38,44,46,44,45,46,44,46,45,45,10]
,[ datetime(2024,6,17),1,4,1,5,10,11,10,10,10,10,10,25,25,-25]
,[ datetime(2024,6,18),1,4,13,47,54,55,55,55,56,54,54,65,65,15]
,[ datetime(2024,6,19),0,4,32,33,41,42,43,43,44,45,44,50,50,5]
,[ datetime(2024,6,20),0,4,11,22,30,33,32,32,31,31,31,40,40,-3]
,[ datetime(2024,6,21),3,4,7,23,29,30,30,30,29,28,28,50,50,5]
,[ datetime(2024,6,22),0,4,23,48,57,58,59,58,60,62,88,82,82,35]
,[ datetime(2024,6,23),4,4,7,9,15,17,17,19,19,20,21,33,33,-18]
,[ datetime(2024,6,24),0,4,12,42,50,53,49,49,51,50,46,55,55,10]
,[ datetime(2024,6,25),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,6,26),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,6,27),None,None,None,None,None,None,None,None,None,None,None,None,None,None]
,[ datetime(2024,6,28),38,43,38,39,47,49,50,49,49,50,50,60,60,10]
,[ datetime(2024,6,29),0,4,26,28,35,38,37,37,37,37,38,55,55,10]
,[ datetime(2024,6,30),0,5,0,7,13,14,14,15,13,12,12,29,29,-14]
,[ datetime(2024,7,1),0,4,29,35,40,41,41,41,42,42,43,50,50,5]
,[ datetime(2024,7,2),0,4,19,23,27,29,30,31,29,31,28,40,40,-5]
,[ datetime(2024,7,3),0,4,24,27,35,34,35,35,35,35,34,128,128,80]
,[ datetime(2024,7,4),0,4,11,22,23,25,26,28,26,24,25,43,43,-3]
,[ datetime(2024,7,5),0,4,11,18,23,26,26,25,25,25,26,45,45,0]
,[ datetime(2024,7,6),21,30,40,46,49,52,55,56,56,55,57,50,50,5]
,[ datetime(2024,7,7),2,4,9,12,13,18,19,21,21,20,21,40,40,-5]
,[ datetime(2024,7,8),0,4,19,22,30,29,31,33,36,36,37,50,50,0]
,[ datetime(2024,7,9),0,5,0,4,7,9,9,11,8,9,15,32,32,-10]
,[ datetime(2024,7,10),1,2,2,1,3,4,22,30,33,31,31,45,45,0]
,[ datetime(2024,7,11),40,46,41,42,45,46,47,50,54,55,55,60,60,10]
,[ datetime(2024,7,12),0,4,23,26,36,42,42,44,50,52,53,60,60,10]
,[ datetime(2024,7,13),31,29,30,33,36,39,38,42,44,44,44,50,50,1]
,[ datetime(2024,7,14),0,4,13,16,17,19,20,24,28,27,27,41,41,-5]
,[ datetime(2024,7,15),1,4,19,23,26,29,29,32,38,40,41,55,55,5]
,[ datetime(2024,7,16),11,30,24,26,32,34,31,31,32,34,34,44,44,0]
,[ datetime(2024,7,17),0,6,0,25,29,30,30,33,35,38,39,55,55,0]
,[ datetime(2024,7,18),69,83,83,87,89,91,92,97,102,102,105,110,110,60]
,[ datetime(2024,7,19),2,4,14,17,18,18,18,20,24,24,23,35,35,-10]
,[ datetime(2024,7,20),0,7,3,3,6,10,14,19,23,24,24,40,40,-5]
,[ datetime(2024,7,21),13,21,17,39,42,46,46,48,50,51,53,55,55,15]
,[ datetime(2024,7,22),2,0,15,15,21,22,20,25,28,29,28,19,19,-8]
,[ datetime(2024,7,23),0,5,4,7,9,9,9,11,11,12,16,35,35,-15]
,[ datetime(2024,7,24),2,8,4,7,8,9,10,11,11,9,13,35,35,-5]
,[ datetime(2024,7,25),4,10,6,8,8,10,12,11,15,17,17,7,7,-35]
,[ datetime(2024,7,26),0,4,7,5,10,14,14,13,17,19,20,41,41,-5]
,[ datetime(2024,7,27),0,6,4,6,6,7,6,9,12,14,14,34,34,-10]
,[ datetime(2024,7,28),0,4,5,6,8,9,10,15,16,20,21,33,33,-13]
,[ datetime(2024,7,29),1,13,4,11,11,13,12,14,12,12,13,32,32,-16]
,[ datetime(2024,7,30),0,7,0,4,5,7,6,10,14,14,18,33,33,0]
,[ datetime(2024,7,31),4,2,2,2,2,2,15,16,22,23,25,45,45,-2]
,[ datetime(2024,8,1),1,2,13,36,38,39,39,41,43,42,42,50,50,10]
,[ datetime(2024,8,2),1,8,2,5,9,9,9,11,12,12,12,30,30,-15]
,[ datetime(2024,8,3),0,4,2,5,6,7,7,8,10,9,9,22,22,-27]
,[ datetime(2024,8,4),1,4,5,7,9,9,8,11,12,12,13,38,38,-6]
,[ datetime(2024,8,5),1,9,5,6,11,11,12,12,14,14,14,35,35,-10]
,[ datetime(2024,8,6),0,4,10,12,15,16,16,16,18,18,19,30,30,-15]
,[ datetime(2024,8,7),0,4,9,12,13,15,16,16,16,18,20,38,38,-5]
,[ datetime(2024,8,8),0,4,16,23,21,22,21,23,27,29,30,50,50,0]
,[ datetime(2024,8,9),25,32,26,27,31,34,35,36,38,39,40,50,50,0]
,[ datetime(2024,8,10),0,8,3,2,4,5,5,7,9,9,9,25,25,-20]
,[ datetime(2024,8,11),0,6,3,5,7,7,6,8,10,9,10,30,30,-18]
,[ datetime(2024,8,12),5,4,8,10,12,12,19,24,28,27,27,45,45,0]
,[ datetime(2024,8,13),0,4,5,6,11,11,12,15,15,16,17,30,30,-15]
,[ datetime(2024,8,14),0,6,5,6,30,41,41,43,45,45,45,55,55,5]
,[ datetime(2024,8,15),0,4,2,6,9,8,8,10,11,11,11,31,31,-17]
,[ datetime(2024,8,16),0,4,17,16,16,16,16,16,18,20,21,40,40,-10]
,[ datetime(2024,8,17),0,27,0,2,2,2,4,2,6,6,6,19,19,-28]
,[ datetime(2024,8,18),0,4,10,13,15,16,20,22,104,109,109,122,122,70]
,[ datetime(2024,8,19),0,4,9,12,14,15,15,17,21,25,28,40,40,-5]
,[ datetime(2024,8,20),0,4,6,8,11,12,13,15,18,22,22,40,40,-8]
,[ datetime(2024,8,21),0,4,0,2,4,3,2,4,4,7,7,39,39,0]
,[ datetime(2024,8,22),8,4,9,12,14,13,11,8,10,19,41,40,40,-3]
,[ datetime(2024,8,23),0,4,1,3,3,4,6,45,55,66,70,67,67,14]
,[ datetime(2024,8,24),0,3,0,3,3,4,4,27,33,47,49,45,45,5]
,[ datetime(2024,8,25),42,50,46,48,48,49,48,50,53,60,63,60,60,14]
,[ datetime(2024,8,26),0,4,0,6,10,11,9,10,12,18,19,30,30,-15]
,[ datetime(2024,8,27),0,5,0,5,5,7,11,13,45,54,55,65,65,15]
,[ datetime(2024,8,28),0,3,0,3,7,8,10,14,27,33,33,45,45,0]
,[ datetime(2024,8,29),15,30,24,26,38,35,30,28,29,32,31,40,40,0]
,[ datetime(2024,8,30),0,72,79,84,84,93,92,92,98,98,98,93,93,68]
,[ datetime(2024,8,31),0,4,0,3,4,4,5,7,9,11,13,25,25,-25]
,[ datetime(2024,9,1),0,1,0,6,9,8,8,13,11,18,17,35,35,-18]
,[ datetime(2024,9,2),0,4,12,15,19,20,20,22,71,107,115,160,160,115]
,[ datetime(2024,9,3),0,7,4,9,9,8,8,12,16,20,27,55,55,9]
,[ datetime(2024,9,4),0,4,5,8,8,14,17,55,59,77,79,68,68,14]
,[ datetime(2024,9,5),0,4,18,22,23,23,23,34,43,51,51,71,71,15]
,[ datetime(2024,9,6),8,10,6,5,9,9,9,67,112,121,123,113,113,89]
,[ datetime(2024,9,7),0,5,1,11,11,13,15,14,15,13,20,33,33,-15]
,[ datetime(2024,9,8),0,3,0,0,1,1,2,2,2,8,8,21,21,-24]
,[ datetime(2024,9,9),0,5,0,6,10,11,10,49,68,76,80,92,92,45]
,[ datetime(2024,9,10),0,2,8,15,15,16,14,16,17,22,22,40,40,-9]
,[ datetime(2024,9,11),0,3,17,27,29,32,36,37,66,71,72,104,104,59]
,[ datetime(2024,9,12),1,6,3,6,8,8,9,12,18,25,29,47,47,5]
,[ datetime(2024,9,13),1,3,0,3,3,3,2,14,17,33,39,52,52,0]
,[ datetime(2024,9,14),1,4,0,4,7,7,9,12,16,21,21,35,35,-10]
,[ datetime(2024,9,15),0,4,6,15,17,19,17,22,25,31,31,51,51,4]
,[ datetime(2024,9,16),0,3,20,25,26,27,26,28,29,37,36,50,50,4]
,[ datetime(2024,9,17),16,21,21,24,25,26,25,28,29,36,35,50,50,0]
,[ datetime(2024,9,18),0,4,5,10,11,9,9,12,13,20,20,35,35,-12]
,[ datetime(2024,9,19),3,6,1,3,10,9,6,9,21,30,32,51,51,5]
,[ datetime(2024,9,20),0,3,0,1,2,2,1,4,5,12,12,36,36,-9]
,[ datetime(2024,9,21),0,3,0,2,2,4,3,5,7,13,9,35,35,-10]
,[ datetime(2024,9,22),26,29,26,47,66,88,99,108,126,141,151,165,165,120]
,[ datetime(2024,9,23),1,6,2,8,8,9,12,13,14,23,25,43,43,-5]
,[ datetime(2024,9,24),0,3,0,7,7,5,8,12,37,45,50,60,60,14]
,[ datetime(2024,9,25),3,4,1,3,4,4,3,10,10,18,18,7,7,-32]
,[ datetime(2024,9,26),26,29,24,26,27,28,28,39,45,54,55,60,60,15]
,[ datetime(2024,9,27),2,6,3,7,9,9,10,13,15,20,21,41,41,-5]
,[ datetime(2024,9,28),1,3,0,0,1,1,5,3,4,12,13,28,28,-20]
,[ datetime(2024,9,29),0,4,0,2,4,5,5,10,11,18,19,40,40,-6]
,[ datetime(2024,9,30),0,5,1,6,8,25,98,101,109,119,120,142,142,99]
,[ datetime(2024,10,1),0,4,5,13,15,17,17,20,26,37,36,40,40,5]
,[ datetime(2024,10,2),0,3,0,8,9,16,16,19,105,113,115,134,134,85]
,[ datetime(2024,10,3),5,4,2,32,34,33,34,35,39,39,39,39,39,0]
,[ datetime(2024,10,4),0,3,0,0,1,2,2,3,6,12,12,35,35,-13]
,[ datetime(2024,10,5),2,5,1,4,6,8,7,12,22,33,34,45,45,5]
,[ datetime(2024,10,6),29,33,28,30,30,32,32,33,47,51,51,55,55,11]
,[ datetime(2024,10,7),2,5,3,5,6,8,8,10,11,16,16,29,29,-20]
,[ datetime(2024,10,8),0,4,0,3,5,6,7,9,12,15,15,34,34,-11]
,[ datetime(2024,10,9),2,8,6,9,13,14,14,20,27,25,26,57,57,10]
,[ datetime(2024,10,10),0,20,15,17,20,20,19,21,24,28,28,45,45,-3]
,[ datetime(2024,10,11),0,3,0,2,2,3,2,4,7,13,12,15,15,-13]
,[ datetime(2024,10,12),1,4,0,1,2,3,3,5,10,16,16,32,32,-15]
,[ datetime(2024,10,13),2,6,0,2,4,5,4,7,11,22,22,28,28,-18]
,[ datetime(2024,10,14),0,3,3,2,4,5,3,3,9,16,12,2,0,-27]
,[ datetime(2024,10,15),0,3,13,22,31,29,31,29,35,38,38,29,29,8]
,[ datetime(2024,10,16),0,4,7,11,11,14,15,48,50,68,73,67,67,15]
,[ datetime(2024,10,17),1,3,0,3,6,6,6,9,17,20,22,41,41,-10]
,[ datetime(2024,10,18),0,3,1,1,0,2,2,0,3,7,6,5,5,-14]
,[ datetime(2024,10,19),1,4,2,7,9,13,12,10,12,16,20,20,20,-2]
,[ datetime(2024,10,20),11,13,6,6,7,7,6,5,5,5,28,45,45,-4]
,[ datetime(2024,10,21),6,22,28,29,30,31,31,33,33,33,37,25,23,-7]
,[ datetime(2024,10,22),0,4,0,7,9,10,11,12,12,13,18,34,34,-10]
,[ datetime(2024,10,23),1,5,7,12,17,18,17,18,18,20,21,39,39,-10]
,[ datetime(2024,10,24),0,4,21,24,28,27,27,27,31,34,37,45,45,0]
,[ datetime(2024,10,25),25,106,106,108,111,110,111,111,109,109,113,125,125,74]
,[ datetime(2024,10,26),0,3,3,3,4,4,4,6,8,8,8,28,28,-17]
,[ datetime(2024,10,27),0,64,59,68,69,70,75,77,77,78,83,97,97,50]
,[ datetime(2024,10,28),6,19,36,32,27,25,58,73,75,76,73,73,73,34]
,[ datetime(2024,10,29),0,22,19,24,26,34,36,40,45,50,52,60,60,14]
,[ datetime(2024,10,30),34,37,35,52,57,68,83,97,112,136,152,174,174,125]
,[ datetime(2024,10,31),0,0,9,9,12,19,23,21,21,28,32,45,45,0]
,[ datetime(2024,11,1),1,4,3,7,9,9,9,11,16,21,23,36,36,-10]
,[ datetime(2024,11,2),0,5,7,10,14,16,15,18,29,45,54,60,60,14]
,[ datetime(2024,11,3),0,3,0,2,5,6,6,9,10,12,24,14,14,-15]
,[ datetime(2024,11,4),46,51,45,49,51,76,76,77,80,85,83,85,85,35]
,[ datetime(2024,11,5),0,3,7,11,12,14,30,32,41,52,50,55,55,10]
,[ datetime(2024,11,6),29,33,29,33,34,36,39,41,43,44,47,54,54,14]
,[ datetime(2024,11,7),0,4,1,6,9,7,9,15,44,52,60,61,61,12]
,[ datetime(2024,11,8),1,4,0,2,4,7,7,11,12,12,20,35,35,-10]
,[ datetime(2024,11,9),0,2,1,3,6,11,13,15,17,20,28,45,45,5]
,[ datetime(2024,11,10),6,20,3,9,12,14,16,17,19,18,26,42,42,-5]
,[ datetime(2024,11,11),0,23,20,25,28,33,37,49,55,72,71,63,63,14]
,[ datetime(2024,11,12),0,5,27,32,32,35,39,42,50,53,53,57,57,14]
,[ datetime(2024,11,13),0,4,20,14,8,13,22,45,55,58,66,58,58,13]
,[ datetime(2024,11,14),1,3,0,3,7,10,15,19,20,21,31,53,53,11]
,[ datetime(2024,11,15),41,42,40,42,44,45,49,53,55,54,56,51,51,23]
,[ datetime(2024,11,16),0,4,5,5,12,18,22,27,32,46,41,63,63,14]
,[ datetime(2024,11,17),0,4,0,7,11,16,16,28,33,47,47,49,49,5]
,[ datetime(2024,11,18),26,28,28,32,35,35,44,48,49,50,58,74,74,34]
,[ datetime(2024,11,19),0,11,8,11,14,17,22,27,49,64,71,66,66,15]
,[ datetime(2024,11,20),5,4,26,37,45,49,53,57,59,141,148,164,164,113]
,[ datetime(2024,11,21),4,4,0,2,4,6,9,14,16,31,32,45,45,-2]
,[ datetime(2024,11,22),10,16,13,13,17,21,29,37,39,39,37,41,41,21]
,[ datetime(2024,11,23),0,4,22,21,26,28,37,40,42,48,55,65,65,12]
,[ datetime(2024,11,24),0,2,0,3,4,8,13,16,19,22,32,26,24,16]
,[ datetime(2024,11,25),0,2,0,4,7,9,13,15,97,109,118,116,116,100]
,[ datetime(2024,11,26),26,82,79,81,86,103,107,109,108,108,118,125,125,75]
,[ datetime(2024,11,27),10,33,30,32,34,46,43,42,43,57,70,71,71,54]
,[ datetime(2024,11,28),1,4,0,1,5,9,13,16,16,15,16,17,17,-1]
,[ datetime(2024,11,29),0,3,0,10,12,14,11,11,11,14,25,40,40,0]
,[ datetime(2024,11,30),0,3,0,2,4,4,20,23,40,41,40,40,40,15]
,[ datetime(2024,11,1),1,4,0,0,5,6,6,7,8,8,17,35,35,-10]
,[ datetime(2024,12,2),0,2,16,15,16,16,16,17,14,19,22,36,36,-12]
,[ datetime(2024,12,3),3,4,3,5,8,9,11,11,12,10,11,30,30,-15]
,[ datetime(2024,12,4),0,4,1,3,5,5,5,5,6,8,11,27,27,-17]
,[ datetime(2024,12,5),0,4,9,13,15,15,14,16,20,32,41,50,50,14]
,[ datetime(2024,12,6),0,4,18,20,22,23,23,24,23,24,32,50,50,0]
,[ datetime(2024,12,7),1,3,19,20,25,26,26,27,27,26,33,50,50,0]
,[ datetime(2024,12,8),0,29,25,28,28,24,28,26,27,26,30,45,45,8]
,[ datetime(2024,12,9),0,4,9,13,15,17,16,20,22,27,37,55,55,11]
,[ datetime(2024,12,10),5,9,6,7,10,12,12,17,16,16,23,14,14,-10]
,[ datetime(2024,12,11),0,21,17,28,27,39,48,59,60,68,71,64,64,14]
,[ datetime(2024,12,12),40,70,41,44,46,47,48,50,53,52,57,60,60,14]
,[ datetime(2024,12,13),0,3,2,2,3,10,10,12,14,17,17,14,14,-4]
,[ datetime(2024,12,14),0,4,0,0,0,2,1,5,6,9,12,30,30,-15]
,[ datetime(2024,12,15),0,1,0,0,4,5,10,10,4,4,4,20,20,-27]
,[ datetime(2024,12,16),0,3,5,4,6,6,7,7,6,6,6,0,0,-19]
   
]
# Prepare the initial table with just dates and delays
dates = [row[0].strftime('%d/%m/%y') for row in data]
df = pd.DataFrame({"Date": dates})

# Add delay data for each station in the format specified
# print(stations)
for i, station in enumerate(stations):
    df[station["label"]] = [row[i + 1] for row in data]

# Define columns in the required order and retrieve weather data
for station in stations:
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": station["longitude"],
        "longitude": station["latitude"],
        "start_date": "2023-12-18",
        "end_date": "2024-12-16",  # Adjusted end date based on data range
        "daily": ["weather_code", "temperature_2m_mean", "wind_speed_10m_max"]
    }

    # Fetch data from API
    response = openmeteo.weather_api(url, params=params)[0]
    print(response)
    daily = response.Daily()
    dates = pd.date_range(
        start=pd.to_datetime(daily.Time(), unit="s", utc=True),
        end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=daily.Interval()),
        inclusive="left"
    )

    # Ensure dates is a list, matching the length of weather data variables
    dates = list(dates)
    weather_code = daily.Variables(0).ValuesAsNumpy()
    temperature = daily.Variables(1).ValuesAsNumpy()
    wind_speed = daily.Variables(2).ValuesAsNumpy()

    # Create a temporary DataFrame to store the data for this station
    temp_df = pd.DataFrame({
        "Date": [date.strftime('%d/%m/%y') for date in dates],
        f"{station['label']}_Delay": df[station["label"]].values,
        f"weather_code_{station['label']}": weather_code,
        f"Temperature_{station['label']}": temperature,
        f"Wind_Speed_{station['label']}": wind_speed,
        f"Distance_travelled_{station['label']}": [None] * len(weather_code)  # Placeholder for Distance
    })

    # Merge this station's data back into the main DataFrame
    df = df.merge(temp_df, on="Date", how="left")

# Drop the original delay columns used for initialization
df.drop(columns=[station["label"] for station in stations], inplace=True)

# Save final table with delays and weather details
df.to_csv("merged_train_weather_data.csv", index=False)
print("Data has been formatted and saved to 'merged_train_weather_data.csv'")

Data has been formatted and saved to 'merged_train_weather_data.csv'
